In [35]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Flatten, Dropout
from tensorflow.keras.layers import Activation, BatchNormalization,Conv2D,MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os


In [3]:
base_dir='data/cat_dog_fulls'
train_dir='data/cat_dog_fulls/train'
test_dir='data/cat_dog_fulls/test'
validation_dir='data/cat_dog_fulls/validation'

In [4]:
train_ds=tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(150,150),
    batch_size=32,
    label_mode='binary'
)
valdiation_ds=tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size=(150,150),
    batch_size=32,
    label_mode='binary'
)
test_ds=tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(150,150),
    batch_size=32,
    label_mode='binary'
)

Found 14000 files belonging to 2 classes.
Found 6000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [5]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_ds = valdiation_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [39]:
IMAGE_SIZE=150
BATCH_SIZE=64
LEARNING_RATE=5e-5 
pretend_net=EfficientNetB4(weights='imagenet',
                           include_top=False,
                           input_shape=(IMAGE_SIZE,IMAGE_SIZE,3))
for layer in pretend_net.layers:
    pretend_net.trainable=False

model=Sequential()
model.add(pretend_net)
model.add(GlobalAveragePooling2D())
model.add(Activation('relu'))
model.add(Dropout(rate=0.2))

model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
             loss='binary_crossentropy',
             metrics=['accuracy'])
call=EarlyStopping(patience=5,
                   verbose=1,
                   monitor='val_loss'
                  )
check=ModelCheckpoint(filepath='./eva.weights.h5',
                      save_best_only=True,
                      save_weights_only=True,
                      monitor='val_accuracy', #좋은거 판단하는 기준 
                      verbose=1
                     )

model.fit(train_ds,
          steps_per_epoch=len(train_ds),
          epochs=30,
          validation_data=validation_ds,
          validation_steps=len(validation_ds),
          verbose=1,
          callbacks=[call,check])

Epoch 1/30
438/438 [==============================] - ETA: 0s - loss: 0.5526 - accuracy: 0.8099   
Epoch 1: val_accuracy improved from -inf to 0.94450, saving model to ./eva.weights.h5
438/438 [==============================] - 45s 90ms/step - loss: 0.5526 - accuracy: 0.8099 - val_loss: 0.4308 - val_accuracy: 0.9445
Epoch 2/30
438/438 [==============================] - ETA: 0s - loss: 0.3605 - accuracy: 0.9471 
Epoch 2: val_accuracy improved from 0.94450 to 0.96200, saving model to ./eva.weights.h5
438/438 [==============================] - 38s 87ms/step - loss: 0.3605 - accuracy: 0.9471 - val_loss: 0.3002 - val_accuracy: 0.9620
Epoch 3/30
438/438 [==============================] - ETA: 0s - loss: 0.2666 - accuracy: 0.9586 
Epoch 3: val_accuracy improved from 0.96200 to 0.96600, saving model to ./eva.weights.h5
438/438 [==============================] - 38s 87ms/step - loss: 0.2666 - accuracy: 0.9586 - val_loss: 0.2307 - val_accuracy: 0.9660
Epoch 4/30
438/438 [========================

In [8]:
# BATCH_SIZE=64
# def eva(x,y):
#     x=tf.cast(x, tf.float32)/255.0
#     return x,y

# train_ds=train_ds.map(eva,num_parallel_calls=AUTOTUNE)
# train_ds=train_ds.cache().shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# validation_ds=validation_ds.map(eva,num_parallel_calls=AUTOTUNE)
# validation_ds=validation_ds.cache().shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# test_ds=test_ds.map(eva,num_parallel_calls=AUTOTUNE)
# test_ds=test_ds.cache().shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
                                 

In [6]:
# from tensorflow.keras.optimizers import RMSprop
# inputs = Input(shape=(150, 150, 3))

# # CNN Block 1
# x =Conv2D(filters=64,kernel_size=(3,3))(inputs)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = MaxPooling2D(pool_size=(2,2))(x)

# # CNN Block 2
# x = Conv2D(filters=128,kernel_size=(3,3))(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = MaxPooling2D(pool_size=(2,2))(x)

# # CNN Block 3
# x = Conv2D(filters=256,
#               kernel_size=(3,3))(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = MaxPooling2D(pool_size=(2,2))(x)

# # FC Layer
# x = Flatten()(x)
# x = Dropout(0.5)(x)
# x = Dense(128)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)

# # 출력층 (이진 분류)
# outputs = Dense(1, activation='sigmoid')(x)

# # 모델 정의
# model = Model(inputs=inputs, outputs=outputs)

# # 컴파일
# model.compile(optimizer=Adam(learning_rate=1e-4),
#               loss='binary_crossentropy',
#               metrics=['accuracy'])

# # 모델 구조 확인
# model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 batch_normalization (Batch  (None, 148, 148, 64)      256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 148, 148, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 128)       73856 

In [7]:
# model.fit(
#     train_ds,
#     validation_data=validation_ds,
#     epochs=10
# )


Epoch 1/10
438/438 [==============================] - 48s 98ms/step - loss: 0.5556 - accuracy: 0.7239 - val_loss: 0.5236 - val_accuracy: 0.7335
Epoch 2/10
438/438 [==============================] - 76s 174ms/step - loss: 0.3935 - accuracy: 0.8221 - val_loss: 0.4425 - val_accuracy: 0.7972
Epoch 3/10
438/438 [==============================] - 27s 63ms/step - loss: 0.3170 - accuracy: 0.8629 - val_loss: 0.5018 - val_accuracy: 0.7837
Epoch 4/10
438/438 [==============================] - 21s 49ms/step - loss: 0.2559 - accuracy: 0.8980 - val_loss: 0.4228 - val_accuracy: 0.8062
Epoch 5/10
438/438 [==============================] - 30s 69ms/step - loss: 0.2102 - accuracy: 0.9169 - val_loss: 0.4540 - val_accuracy: 0.8003
Epoch 6/10
438/438 [==============================] - 22s 49ms/step - loss: 0.1722 - accuracy: 0.9391 - val_loss: 0.4897 - val_accuracy: 0.7885
Epoch 7/10
438/438 [==============================] - 21s 49ms/step - loss: 0.1455 - accuracy: 0.9494 - val_loss: 0.4322 - val_accuracy

In [8]:
from tensorflow.keras.applications import VGG16  # 예시로 VGG16 사용
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization, Activation
from tensorflow.keras.models import Model

# 사전 학습된 VGG16 모델을 불러옵니다 (ImageNet 데이터셋으로 학습된 모델)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Fine-tuning을 위한 모델 구성
inputs = Input(shape=(150, 150, 3))

# VGG16의 베이스 모델을 고정 (학습되지 않도록)
x = base_model(inputs)
x = Flatten()(x)  # 출력층으로 플래튼

# FC Layer
x = Dropout(0.5)(x)
x = Dense(128)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# 출력층 (이진 분류)
outputs = Dense(1, activation='sigmoid')(x)

# 전체 모델 정의
model = Model(inputs=inputs, outputs=outputs)

# Fine-tuning: VGG16 모델의 가중치는 동결하고, 마지막 Dense 레이어만 학습
for layer in base_model.layers:
    layer.trainable = False  # VGG16의 모든 레이어를 학습되지 않도록 설정

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 모델 구조 확인
model.summary()

# 모델 훈련
model.fit(train_ds, validation_data=validation_ds, epochs=10)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dropout_1 (Dropout)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               1048704   
                                                                 
 batch_normalization_4 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                           